In [15]:
!pip install tslearn
import numpy as np
import pandas as pd
from tslearn.datasets import UCR_UEA_datasets
from tslearn.metrics import dtw


In [16]:
def lcss_distance(x1, x2, epsilon):

    n = len(x1)
    m = len(x2)
    lcss_matrix = np.zeros((n + 1, m + 1))

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if abs(x1[i - 1] - x2[j - 1]) <= epsilon:
                lcss_matrix[i, j] = lcss_matrix[i - 1, j - 1] + 1
            else:
                lcss_matrix[i, j] = max(lcss_matrix[i - 1, j], lcss_matrix[i, j - 1])

    lcss_distance = 1 - (lcss_matrix[n, m] / min(n, m))
    return lcss_distance


In [17]:
def dtw_distance(x, y):
    n = len(x)
    m = len(y)

    # ساخت ماتریس فاصله
    distance_matrix = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            distance_matrix[i, j] = abs(x[i] - y[j])

    # ساخت ماتریس مسیر
    path_matrix = np.zeros((n, m))
    path_matrix[0, 0] = distance_matrix[0, 0]
    for i in range(1, n):
        path_matrix[i, 0] = distance_matrix[i, 0] + path_matrix[i-1, 0]
    for j in range(1, m):
        path_matrix[0, j] = distance_matrix[0, j] + path_matrix[0, j-1]
    for i in range(1, n):
        for j in range(1, m):
            path_matrix[i, j] = distance_matrix[i, j] + min(path_matrix[i-1, j], path_matrix[i, j-1], path_matrix[i-1, j-1])

    # محاسبه فاصله DTW
    dtw_distance = path_matrix[n-1, m-1]

    return dtw_distance

In [18]:
def euclidean_distance(x, y):
    return np.linalg.norm(x - y)

def manhattan_distance(x, y):
    return np.sum(np.abs(x - y))

In [19]:
# لود دیتاست مورد نظر از UCR
dataset_name = 'Wafer'
X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)

In [20]:
def gaussian_kernel_value(time_series1, time_series2 , metric : str , sigma = 0.5):
    gama = 1 / (2*sigma**2)
    if metric == 'ED':
        return np.exp(-gama * euclidean_distance(time_series1, time_series2))
    if metric == 'MD':
        return np.exp(-gama * manhattan_distance(time_series1, time_series2))
    if metric == 'DTW':
        return np.exp(-gama * dtw_distance(time_series1, time_series2))
    if metric == 'LCSS':
        return np.exp(-gama * lcss_distance(time_series1, time_series2,0.5))

In [23]:
def KernelMatrix(X_train , metric : str):

    n = len(X_train)
    kernel_matrix = np.zeros((n,n))

    for i in range(n):
        for j in range(i+1, n):
            time_series1 = X_train[i]
            time_series2 = X_train[j]

            kernel_matrix[i][j] = gaussian_kernel_value( time_series1  ,  time_series2 ,  metric , 0.4)
            kernel_matrix[j][i] = kernel_matrix[i][j]

    return kernel_matrix

In [28]:
pd.DataFrame(KernelMatrix(X_train , 'MD'))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000e+00,1.302308e-319,9.404159e-230,2.269508e-164,3.025906e-167,1.371861e-254,2.831743e-316,7.152282e-310,4.018284e-106,1.029605e-171,...,4.105258e-18,2.887660e-157,2.982936e-16,7.827725e-166,1.690636e-280,9.094200e-17,5.855253e-298,3.023215e-173,1.010786e-292,4.895781e-160
1,1.302308e-319,0.000000e+00,9.465009e-188,0.000000e+00,0.000000e+00,4.703652e-165,3.342275e-20,2.166032e-51,0.000000e+00,1.311924e-279,...,1.406482e-316,0.000000e+00,7.460391e-322,0.000000e+00,6.479689e-124,3.359646e-322,2.298517e-111,6.418135e-313,6.128253e-121,0.000000e+00
2,9.404159e-230,9.465009e-188,0.000000e+00,5.144128e-263,2.621483e-270,3.038803e-64,3.607901e-183,5.227826e-174,3.343597e-270,1.342795e-189,...,6.902075e-228,9.337650e-272,3.928638e-229,1.267329e-282,2.017004e-115,2.719778e-229,3.584361e-133,5.214361e-212,2.347911e-126,1.866847e-266
3,2.269508e-164,0.000000e+00,5.144128e-263,0.000000e+00,1.476285e-25,2.736359e-292,0.000000e+00,0.000000e+00,1.511448e-107,8.674321e-208,...,2.763588e-171,3.172988e-48,2.330072e-158,2.422815e-66,0.000000e+00,6.040297e-158,0.000000e+00,6.866975e-153,0.000000e+00,1.219702e-29
4,3.025906e-167,0.000000e+00,2.621483e-270,1.476285e-25,0.000000e+00,1.492471e-303,0.000000e+00,0.000000e+00,4.584529e-97,6.544137e-220,...,1.632491e-174,3.985508e-30,4.025921e-161,1.295500e-46,0.000000e+00,9.958791e-161,0.000000e+00,3.380805e-170,0.000000e+00,2.981790e-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9.094200e-17,3.359646e-322,2.719778e-229,6.040297e-158,9.958791e-161,1.100876e-255,4.338193e-319,2.395953e-305,3.698790e-110,1.898124e-167,...,4.953782e-33,2.299267e-161,5.692968e-02,1.001142e-167,7.926118e-285,0.000000e+00,1.124620e-301,1.268988e-172,2.629299e-289,3.182036e-155
996,5.855253e-298,2.298517e-111,3.584361e-133,0.000000e+00,0.000000e+00,1.714680e-102,4.097068e-102,8.199337e-105,0.000000e+00,6.791203e-226,...,5.661855e-294,0.000000e+00,1.735486e-301,0.000000e+00,1.892102e-65,1.124620e-301,0.000000e+00,1.865477e-251,3.864148e-45,0.000000e+00
997,3.023215e-173,6.418135e-313,5.214361e-212,6.866975e-153,3.380805e-170,1.219453e-226,3.972601e-300,8.497120e-286,4.670365e-191,7.147133e-94,...,4.688958e-175,4.253035e-180,1.589621e-172,3.917494e-196,3.199564e-255,1.268988e-172,1.865477e-251,0.000000e+00,1.154332e-238,8.788704e-168
998,1.010786e-292,6.128253e-121,2.347911e-126,0.000000e+00,0.000000e+00,1.291400e-105,3.773792e-112,4.070846e-98,0.000000e+00,1.222925e-212,...,2.369167e-296,0.000000e+00,4.001514e-289,0.000000e+00,1.230618e-84,2.629299e-289,3.864148e-45,1.154332e-238,0.000000e+00,0.000000e+00


In [26]:
pd.DataFrame(KernelMatrix(X_train , 'ED'))

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.000000e+00,1.133270e-28,9.579062e-26,2.168348e-17,8.740592e-18,1.117881e-26,2.031844e-28,2.673361e-28,7.195836e-15,5.862338e-22,...,4.172804e-06,2.032018e-17,8.934430e-06,2.500441e-18,1.116979e-27,8.537389e-06,3.069029e-28,5.282095e-19,2.687821e-28,1.609989e-17
1,1.133270e-28,0.000000e+00,4.717165e-23,2.397551e-33,8.934630e-33,2.488706e-21,2.438745e-06,1.098735e-09,2.657929e-30,5.129731e-26,...,1.411949e-28,3.065611e-32,9.266682e-29,1.093985e-31,4.438836e-18,8.863394e-29,9.455241e-17,1.114793e-28,1.473554e-17,9.158202e-33
2,9.579062e-26,4.717165e-23,0.000000e+00,8.375486e-26,1.388735e-26,4.789766e-19,8.024077e-23,2.292105e-22,7.310214e-27,2.529106e-22,...,9.207175e-26,1.391043e-26,1.358029e-25,7.623987e-28,5.835121e-23,1.336846e-25,2.802258e-22,4.178502e-23,1.050844e-21,4.013687e-26
3,2.168348e-17,2.397551e-33,8.375486e-26,0.000000e+00,1.589895e-07,2.487646e-27,2.060892e-33,1.531297e-32,2.872527e-13,3.250268e-23,...,1.558951e-17,8.583200e-10,3.139277e-17,5.399809e-12,6.097850e-31,3.375755e-17,6.789160e-31,1.333730e-19,2.682610e-30,1.953206e-07
4,8.740592e-18,8.934630e-33,1.388735e-26,1.589895e-07,0.000000e+00,1.244581e-28,2.753817e-33,1.004479e-32,9.901032e-12,2.703435e-24,...,5.302292e-18,5.512564e-07,1.308871e-17,1.953260e-09,7.229024e-31,1.413696e-17,4.915243e-32,4.185077e-21,1.054029e-31,3.371746e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8.537389e-06,8.863394e-29,1.336846e-25,3.375755e-17,1.413696e-17,1.322314e-26,1.519603e-28,5.409037e-28,2.290243e-15,9.630538e-22,...,1.556442e-08,6.085678e-18,4.229992e-01,1.475896e-18,8.718154e-28,0.000000e+00,2.432872e-28,2.825186e-19,4.941769e-28,2.643850e-17
996,3.069029e-28,9.455241e-17,2.802258e-22,6.789160e-31,4.915243e-32,1.338894e-20,5.841531e-16,3.349475e-16,4.048037e-31,8.463867e-23,...,3.498502e-28,5.597098e-32,2.543000e-28,1.556259e-31,2.609227e-18,2.432872e-28,0.000000e+00,3.208882e-25,1.109206e-10,8.902887e-32
997,5.282095e-19,1.114793e-28,4.178502e-23,1.333730e-19,4.185077e-21,7.498880e-24,6.259358e-28,4.339132e-27,5.407322e-23,6.300254e-14,...,8.887738e-19,7.880823e-22,2.986044e-19,2.801321e-23,9.731815e-26,2.825186e-19,3.208882e-25,0.000000e+00,2.050153e-24,8.762513e-21
998,2.687821e-28,1.473554e-17,1.050844e-21,2.682610e-30,1.054029e-31,4.016563e-20,7.552720e-17,9.747080e-16,5.836671e-32,8.178664e-22,...,1.201326e-28,3.700837e-32,5.126507e-28,4.041544e-32,1.039193e-18,4.941769e-28,1.109206e-10,2.050153e-24,0.000000e+00,6.996523e-31


In [ ]:
pd.DataFrame(KernelMatrix(X_train , 'LCSS'))

In [ ]:
pd.DataFrame(KernelMatrix(X_train , 'DTW'))